In [1]:
import pandas as pd
import numpy as np
from rhessys import utilities as rut

## File Header:
 Niwot (663)
 Colorado  SNOTEL Site - 9910 ft
 Daily Data for Period of Record

 (As of: Mon Jul 06 12:41:04 PDT 2015)
 
  **Provisional data, subject to revision**

Date,Air Temperature Average (degC),Air Temperature Maximum (degC),Air Temperature Minimum (degC),Precipitation Accumulation (mm),Precipitation Increment (mm),Precipitation Increment - Snow-adj (mm),Snow Depth (cm),Snow Water Equivalent (mm)

In [2]:
header = ['date','Tave','Tmax','Tmin','AccumP','P','Psnowadj','SD','SWE']

In [3]:
data = pd.read_csv('./data/SNOTEL/niwot/niwot.csv',skiprows=7,
                   header=False,names=header)
data.index = pd.DatetimeIndex(data.date)
del data['date']

In [4]:
data.head()

,Tave,Tmax,Tmin,AccumP,P,Psnowadj,SD,SWE
1980-10-01,NaN,NaN,NaN,0,0,0,NaN,0
1980-10-02,NaN,NaN,NaN,0,0,0,NaN,0
1980-10-03,NaN,NaN,NaN,0,0,0,NaN,0
1980-10-04,NaN,NaN,NaN,0,0,0,NaN,0
1980-10-05,NaN,NaN,NaN,0,0,0,NaN,0


# Compute Snowmelt

In [5]:
# compute snowmelt
sn1 = data.SWE.as_matrix()[:-1] # grab the first through the 2nd to last measurement
sn2 = data.SWE.as_matrix()[1:] # grab the second through the last measurement

sm = sn1-sn2 # compute melt
# add a nan to the end to shift the snowmelt to the correct position in the timeseries
sm = np.concatenate([sm,[np.NaN]],axis=1) 

data['snowmelt'] = sm # put the time series in the data frame

## Shift the temp data up so that it cooresponds to the day it was collected

In [6]:
header = list(data.columns)

In [7]:
tmp = pd.DataFrame(columns=header,index=[(data.index[0]-pd.datetools.timedelta(1))])

In [8]:
data = pd.concat([tmp,data])

In [9]:
cols = ['Tave','Tmin','Tmax']

for col in cols:
    data.ix[0:-1,col] = data.ix[1:,col] # scoot up the time series
    data.ix[-1,col] = np.NaN # make the last value nan

## Compute the Water Year and Day of Water Year

In [10]:
data['wateryear'] = data.index.map(rut.wateryear)
data['dowy'] = data.index.map(rut.DOWY)

## Save the Data

In [13]:
data.to_pickle('./data/niwot_new.pcl')